# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry
```

## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit


- Import simulator

In [2]:
from qiskit import BasicAer

backend_qasm = BasicAer.get_backend('qasm_simulator')

- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [8]:
from qurry.recipe import TrivialParamagnet, GHZ

In [9]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [10]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
    3: ...
    4: ...
    5: ...
} with 6 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [11]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
    backend=backend_qasm
)
exp1 

'a266d8bc-60be-4652-92b8-f27edf9e6c85'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`

In [12]:
exp2 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp2

'cf312754-153e-49b9-8caa-48d21d7990b8'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [13]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    a266d8bc-60be-4652-92b8-f27edf9e6c85: ...
    cf312754-153e-49b9-8caa-48d21d7990b8: ...
} with 2 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [14]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=a266d8bc-60be-4652-92b8-f27edf9e6c85, EntropyRandomizedArguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='a266d8bc-60be-4652-92b8-f27edf9e6c85', wave_key='TrivialParamagnet', shots=1024, backend=<QasmSimulatorPy('qasm_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-03-28 15:42:04', 'run': '2024-03-28 15:42:05'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [15]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=a266d8bc-60be-4652-92b8-f27edf9e6c85
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ a266d8bc-60be-4652-92b8-f27edf9e6c85   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- qasm_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

### Bit select hint

A bit string mapping to the index of qubit is `'76543210'` for 8 qubits example here.

| the input | bitStringRange | String Slice | Actual Qubit Included |
| --- | --- | --- | --- |
| `2` | `(6, 8)` | `'10'` | 01 |
| `4` | `(4, 8)` | `'3210'` | 0123 |
| `6` | `(2, 8)` | `'543210'` | 012345 |
| `(2, 4)` | `(2, 4)` | `'54'` | 45 |
| `(4, 6)` | `(4, 6)` | `'32'` | 23 |
| `(2, 6)` | `(2, 6)` | `'5432'` | 2345 |
| `(-2, 2)` | `(-2, 2)` | `'1076'` | 6701 |
| `(-4, 2)` | `(-4, 2)` | `'321076'` | 670123 |
| `(-2, 4)` | `(-2, 4)` | `'107654'` | 456701 |
| `3` | `(5, 8)` | `'210'` | 012 |
| `(5, 7)` | `(5, 7)` | `'21'` | 12 |
| `(4, 7)` | `(4, 7)` | `'321'` | 123 |


In [22]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(3, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=1
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [17]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.9992536735534668, entropy=0.0010771234561738688, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-28 15:43:32', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [18]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.9992536735534668, entropy=0.0010771234561738688, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-28 15:43:32', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [19]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [20]:
main

{'purity': 0.9992536735534668,
 'entropy': 0.0010771234561738688,
 'puritySD': 0.8902214609811956,
 'entropySD': 1.2852773235081623,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.1773760414123535,
 'entropyAllSys': -0.23557517546139203,
 'puritySDAllSys': 1.356173409325207,
 'entropySDAllSys': 1.6617839869341386,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.08539008367112783,
 'mitigatedPurity': 0.8576585003250091,
 'mitigatedEntropy': 0.22152478045088284,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.00083155,
 'takingTimeAllSys': 0.003305909,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-03-28 15:43:32',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [21]:
side_prodct

{'purityCells': {37: 0.4309425354003906,
  69: 0.26760101318359375,
  75: 1.4965877532958984,
  80: 1.8793811798095703,
  77: 0.17938995361328125,
  27: 2.089414596557617,
  30: 0.22678565979003906,
  34: 0.3164329528808594,
  72: 2.457324981689453,
  40: 0.11249351501464844,
  92: 0.17421722412109375,
  10: 0.3518810272216797,
  8: 0.8922939300537109,
  29: 0.4043235778808594,
  36: 0.2643165588378906,
  12: 0.9281597137451172,
  25: 0.2592754364013672,
  61: 2.97021484375,
  73: 1.7795886993408203,
  81: 0.7441387176513672,
  90: 0.9074287414550781,
  84: 0.8696174621582031,
  91: 1.2699203491210938,
  98: 0.12272453308105469,
  6: 0.6167259216308594,
  41: 1.560943603515625,
  44: 4.94427490234375,
  68: 0.2623481750488281,
  54: 0.7764453887939453,
  70: 1.1307144165039062,
  79: 1.106790542602539,
  89: 0.6030216217041016,
  28: 3.3140697479248047,
  96: 0.8483428955078125,
  97: 0.7578659057617188,
  43: 0.9579372406005859,
  16: 1.2248191833496094,
  22: 0.18966102600097656,
  6

In [19]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-03-19 19:05:47
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (0, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.1818041610717773
   - entropy ---------------------- -0.2409909836888523
   - puritySD --------------------- 1.1137335938579271
   - entropySD -------------------- 1.3595974575627563
   - purityCells ------------------ {61: 1.3342533111572266, 25: 4.869670867919922, 29: 1.5328655242919922, 65: 3.6087779998779297, 70: 0.8376827239990234, 35: 0.7242488861083984, 72: 1.8999004364013672, 16: 0.35748291015625, 56: 0.1423511505126953, 85: 2.0770034790039062, 20: 0.9743423461914062, 98: 0.5528850555419922, 93: 3.

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.

In [20]:
from qurry.process import AVAIBILITY_STATESHEET
print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.7.1.dev5
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability           Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

